In [1]:
import pandas as pd

In [2]:
Pensionados = pd.read_excel('C:/Users/HP/Downloads/PENSIONES_proyecto1.xlsx','informacion')
mejoras = pd.read_excel('C:/Users/HP/Downloads/PENSIONES_proyecto1.xlsx','mejoras')
mortalidad=pd.read_excel('C:/Users/HP/Downloads/PENSIONES_proyecto1.xlsx','mortalidad')

Para calcular cada anualidad:

In [9]:
def Anualidad(id):
    Edad=Pensionados['Edad_Actual'][id]
    Sexo=Pensionados['Sexo'][id]
    Factor_de_ajuste=(24-1)/(2*24)
    K=[]
    for i in range(111):
        K.append(i)
    t=[]
    for i in range(111):
        t.append(i+10)
    Vk=[]
    for i in range(111):
        Vk.append((1.06)**(-i))
    I=[]
    for i in range(111):
        I.append((1.03)**(i))
    if Sexo =='Masculino':
        Px_t=[]
        for i in range(111):
            if i < (111-Edad):
                Px_t.append(1-mortalidad['Hombres_qx'][Edad+i]*(1-mejoras['Hombres_TMx'][Edad+i])**(i+11))
            else:
                Px_t.append(1)
    else:
        Px_t=[]
        for i in range(111):
            if i < (111-Edad):
                Px_t.append(1-mortalidad['Mujeres_qx'][Edad+i]*(1-mejoras['Mujeres_TMx'][Edad+i])**(i+11))
            else:
                Px_t.append(1)
    kPx=[1]
    for i in range(111-1):
        kPx.append(kPx[i]*Px_t[i])
    V=[]
    for i in range(111):
        V.append(Vk[i]*kPx[i])
    A=[]
    for i in range(111):
        A.append(V[i]*I[i])
    df = pd.DataFrame({'K' : K,'t' : t,'V^k' : Vk, 'Inflación' : I,'Px+t' : Px_t, 'kPx' : kPx, 'Anualidad sin inflación': V, 
                   'Anualidad con inflación': A})
    a=sum(A)
    Anualidad_ajustada=a-Factor_de_ajuste
    #cuidado con la siguiente linea, solo correr cuando ya sw quieran descargar las 100 anualidades
    #df.to_excel(r'C:\Users\HP\Documents\AnualidadId'+str(id)+'.xlsx',  sheet_name='Id'+str(id), index = False)
    return(Anualidad_ajustada)
    #return(df)
    




In [11]:
Anualidad(0)


13.298188602985709

Para calcular las anualidades de cada ID y apendizarlas en un vector:

In [149]:
Anualidades=[]
for i in range(100):
    Anualidades.append(Anualidad(i))
df=pd.DataFrame(Anualidades)

Para guardar el vector anualidades

In [12]:
df.to_excel(r'C:\Users\HP\Documents\Vector_anualidades.xlsx',  sheet_name='Vector_anualidades', index = False)

In [75]:
def Anualidad_con_faltantes(id):
    Faltantes=Pensionados['Años faltantes de garantia'][id]
    Edad=Pensionados['Edad_Actual'][id]
    Sexo=Pensionados['Sexo'][id]
    Factor_de_ajuste=(24-1)/(2*24)
    K=[]
    for i in range(111):
        K.append(i)
    t=[]
    for i in range(111):
        t.append(i+10+Faltantes)
    Vk=[]
    for i in range(111):
        Vk.append((1.06)**(-i))
    I=[]
    for i in range(111):
        I.append((1.03)**(i))
    if Sexo =='Masculino':
        Px_t=[]
        for i in range(111):
            if i < (111-Edad-Faltantes):
                Px_t.append(1-mortalidad['Hombres_qx'][Edad+i+Faltantes]*(1-mejoras['Hombres_TMx'][Edad+i+Faltantes])**(i+11+Faltantes))
            else:
                Px_t.append(1)
    else:
        Px_t=[]
        for i in range(111):
            if i < (111-Edad-Faltantes):
                Px_t.append(1-mortalidad['Mujeres_qx'][Edad+i+Faltantes]*(1-mejoras['Mujeres_TMx'][Edad+i+Faltantes])**(i+11+Faltantes))
            else:
                Px_t.append(1)
    kPx=[1]
    for i in range(111-1):
        kPx.append(kPx[i]*Px_t[i])
    V=[]
    for i in range(111):
        V.append(Vk[i]*kPx[i])
    A=[]
    for i in range(111):
        A.append(V[i]*I[i])
    a=sum(A)
    Anualidad_ajustada=a-Factor_de_ajuste
    
    df = pd.DataFrame({'K' : K,'t' : t,'V^k' : Vk, 'Inflación' : I,'Px+t' : Px_t, 'kPx' : kPx, 'Anualidad sin inflación': V, 
                   'Anualidad con inflación': A})
    
    #cuidado con la siguiente linea, solo correr cuando ya sw quieran descargar las 100 anualidades
    #df.to_excel(r'C:\Users\HP\Documents\AnualidadId'+str(id)+'.xlsx',  sheet_name='Id'+str(id), index = False)
    return(Anualidad_ajustada)
    #return(df)

In [136]:
Anualidades2=[]
for i in range(100):
    Anualidades2.append(Anualidad_con_faltantes(i))


In [193]:
def Anualidad_Diferida(id):
    Sexo=Pensionados['Sexo'][id]
    Faltantes=Pensionados['Años faltantes de garantia']
    Edad=Pensionados['Edad_Actual']
    j=1.06/1.03
    Anualidad_diferida=[]
    if Faltantes[id] == 0:
        Anualidad_diferida.append(0)
    else:
        if Sexo =='Masculino':
            Px_t=[]
            for i in range(111):
                if i < (111-Edad):
                    Px_t.append(1-mortalidad['Hombres_qx'][Edad+i]*(1-mejoras['Hombres_TMx'][Edad+i])**(i+11))
                else:
                    Px_t.append(1)
        else:
            Px_t=[]
            for i in range(111):
                if i < (111-Edad):
                    Px_t.append(1-mortalidad['Mujeres_qx'][Edad+i]*(1-mejoras['Mujeres_TMx'][Edad+i])**(i+11))
                else:
                    Px_t.append(1)
    kPx=[1]
    for i in range(111-1):
        kPx.append(kPx[i]*Px_t[i])
        
    for i in range(100):
        Anualidad_diferida.append(Anualidades2[i]*kPx[Faltantes[i]]*((j)**(-Faltantes[i])))
    df2=pd.DataFrame(Anualidad_diferida)
    return(Anualidad_diferida)

        


In [151]:
df2.to_excel(r'C:\Users\HP\Documents\Vector_anualidades_diferidas.xlsx',  sheet_name='Vector_anualidades_diferidas', index = False)

A partir de aquí solo es código que sirvió para construir la función, lo dejo por si quieren checar parte por parte

In [88]:
K=[]
for i in range(111):
    K.append(i)

In [89]:
t=[]
for i in range(111):
    t.append(i+10)

In [90]:
Vk=[]
for i in range(111):
    Vk.append((1.06)**(-i))

In [91]:
I=[]
for i in range(111):
    I.append((1.03)**(i))

In [92]:
Px_t=[]
for i in range(111):
    if i < (111-71):
        Px_t.append(1-mortalidad['Hombres_qx'][71+i]*(1-mejoras['Hombres_TMx'][71+i])**(i+10))
    else:
        Px_t.append(1)

In [93]:
kPx=[1]
for i in range(111-1):
    kPx.append(kPx[i]*Px_t[i])


In [94]:
V=[]
for i in range(111):
    V.append(Vk[i]*kPx[i])

In [95]:
A=[]
for i in range(111):
    A.append(V[i]*I[i])

In [16]:
df = pd.DataFrame({'K' : K,'t' : t,'V^k' : Vk, 'Inflación' : I,'Px+t' : Px_t, 'kPx' : kPx, 'Anualidad sin inflación': V, 
                   'Anualidad con inflación': A})
df

,K,t,V^k,Inflación,Px+t,kPx,Anualidad sin inflación,Anualidad con inflación
0,0,10,1.000000,1.000000,0.979923,1.000000,1.000000,1.000000
1,1,11,0.943396,1.030000,0.978723,0.979923,0.924456,0.952190
2,2,12,0.889996,1.060900,0.977415,0.959074,0.853572,0.905555
3,3,13,0.839619,1.092727,0.976010,0.937413,0.787070,0.860052
4,4,14,0.792094,1.125509,0.974488,0.914925,0.724706,0.815663
5,5,15,0.747258,1.159274,0.972927,0.891583,0.666243,0.772358
6,6,16,0.704961,1.194052,0.971246,0.867445,0.611514,0.730180
7,7,17,0.665057,1.229874,0.969441,0.842502,0.560312,0.689113
8,8,18,0.627412,1.266770,0.967512,0.816756,0.512443,0.649147
9,9,19,0.591898,1.304773,0.965463,0.790222,0.467731,0.610283


In [17]:
Anualidad=sum(A)

In [18]:
Factor_de_ajuste=(24-1)/(2*24)

In [19]:
Anualidad_ajustada=Anualidad-Factor_de_ajuste
Anualidad_ajustada

13.269932934204943